## 2. Calculate Adjusted Odds Ratios
Disclaimer:
- The dataset shared in this notebook is a **synthetic dataset** created for demonstration purposes only.
- All results presented in our paper are based on the Merative™ MarketScan® Databases, which contain de-identified real-world healthcare data and cannot be publicly shared.
- For details about the actual maternal and neonatal datasets, please refer to the **Data Availability** and **Methods** sections of our paper.
 

In [1]:
import os
import sys
import warnings
# Function to set thread limits for external libraries to avoid oversubscription in the shared server
def set_threads_for_external_libraries(n_threads=1):
    if ("numpy" in sys.modules) or ("scipy" in sys.modules) or ("sklearn" in sys.modules):
        warnings.warn("Call set_threads_for_external_libraries() before importing numpy/scipy/sklearn for full effect.")
    for k in ["OMP_NUM_THREADS","OPENBLAS_NUM_THREADS","MKL_NUM_THREADS","VECLIB_MAXIMUM_THREADS","NUMEXPR_NUM_THREADS"]:
        os.environ[k] = str(n_threads)
        
set_threads_for_external_libraries(n_threads=64)

In [2]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import PerfectSeparationError
import time
from sklearn.preprocessing import MinMaxScaler
from numpy.linalg import inv, det
from category_encoders.binary import BinaryEncoder
import statsmodels.formula.api as smf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import optuna
import time
import logging

In [3]:
## Select the list of medications and disease pairs which already showed statistically significant raw (unadjusted) odds ratios.
list_to_test = pd.read_csv("../0_data/results/unadjusted_odds_results.csv")  # Columns: medication, disease
med_dz_pair = list_to_test[['Disease','Medication']]


In [4]:
## Load the main data
df = pd.read_csv('../0_data/synthetic_baby_mom_data.csv')
df = df.set_index('ENROLID_BABY')
df

,SEX,REGION,LOCATION,GESTATIONAL_AGE,AGE_MOM,RDS_Baby,NAS_Baby,Postmaturity_Baby,ROP_Baby,SGA_Baby,...,ADHD_Mom,Depression_Mom,Eclampsia_Mom,Epilepsy_Mom,Infertility_Mom,GDM_Mom,"Ondansetron, Oral","Sertraline, Oral","Oxycodone, Oral","Acetaminophen, Oral"
ENROLID_BABY,,,,,,,,,,,,,,,,,,,,,
ENR_8AF1F,1,3,46,37,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENR_756BE,1,5,19,38,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENR_F24D4,1,4,35,39,31,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENR_E1CBF,1,4,30,38,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENR_7D976,1,4,1,41,31,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENR_8B740,2,2,6,38,31,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENR_DD40A,1,2,36,39,31,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENR_26F00,2,3,42,39,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
med_dz_pair = med_dz_pair.loc[:1,:]
med_dz_pair


,Disease,Medication
0,RDS_Baby,"Ondansetron, Oral"


In [6]:
# Prepare dataset for adjusted odds ratio calculations
# encoding categorical variables, scaling continuous variables, and drop unnecessary columns
# Please modify this part based on your dataset structure
def prepare_odds_data(
    df: pd.DataFrame,
    outcome_suffix: str = "_Baby",
    mom_suffix: str = "_Mom",
    med_start_col: str = "Ondansetron, Oral",
    scaler_cols=("GESTATIONAL_AGE", "AGE_MOM"),
    categorical_cols=("REGION", "LOCATION"),
    drop_cols=("REGION", "SUD_Psystimul_Amphetamine_Mom"), #Preselected based on VIF analysis
):
    #apply binary encoding to location columns (categorical variable)
    if "LOCATION" in df.columns:
        enc = BinaryEncoder(cols=["LOCATION"], drop_invariant=True)
        df = enc.fit_transform(df)        
    
    #Divide the dataframe into different parts 
    baby_cols = [c for c in df.columns if c.endswith(outcome_suffix)]
    mom_dz_cols = [c for c in df.columns if c.endswith(mom_suffix)]
    
    baby_dz = df[baby_cols]
    mom_dz = df[mom_dz_cols]
    
    others_df = df.loc[:,:'AGE_MOM'] # modify as needed based on column order.
    
    if med_start_col not in df.columns:
        raise KeyError(f"med_start_col '{med_start_col}' not in df.columns")
    mom_med = df.loc[:,med_start_col:] # modify as needed based on column order
    
    #scale gestational age and maternal age
    scaler = MinMaxScaler()
    norm_other_df = others_df.copy().reset_index()
    scaler.fit(norm_other_df[['GESTATIONAL_AGE']])
    norm_other_df[['GESTATIONAL_AGE']]=pd.DataFrame(scaler.transform(norm_other_df[['GESTATIONAL_AGE']]),columns=['GESTATIONAL_AGE'])
    scaler.fit(norm_other_df[['AGE_MOM']])
    norm_other_df[['AGE_MOM']]=pd.DataFrame(scaler.transform(norm_other_df[['AGE_MOM']]),columns=['AGE_MOM'])
    norm_other_df = norm_other_df.set_index('ENROLID_BABY')
    
    #combine confounder dataframes
    confounders = pd.concat([norm_other_df,mom_dz],axis=1)
    
    ## 3. Drop null columns, REGION and amphetamine columns based on VIF
    confounders_corr = confounders.corr().abs()
    nan_col = list(confounders_corr[confounders_corr['GESTATIONAL_AGE'].isna()].index)
    confounders = confounders.drop(columns=nan_col,axis=1)
    confounders = confounders.drop(columns=['REGION']) #,'SUD_Psystimul_Amphetamine_Mom'
    odds_df = pd.concat([baby_dz,confounders,mom_med],axis=1)

    return odds_df, confounders

In [7]:
odds_df, confounders =prepare_odds_data(
                                        df,
                                        outcome_suffix = "_Baby",
                                        mom_suffix = "_Mom",
                                        med_start_col = "Ondansetron, Oral",
                                        scaler_cols=("GESTATIONAL_AGE", "AGE_MOM"),
                                        categorical_cols=("REGION", "LOCATION"),
                                        drop_cols=("REGION"), #, "SUD_Psystimul_Amphetamine_Mom"
                                    )

/home/ykim824/miniconda3/envs/pregmednet/lib/python3.10/site-packages/category_encoders/ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
/home/ykim824/miniconda3/envs/pregmednet/lib/python3.10/site-packages/category_encoders/ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
/home/ykim824/miniconda3/envs/pregmednet/lib/python3.10/site-packages/category_encoders/ordinal.py:198: FutureWarning:

In [8]:
## Prepare confounder formula string for statsmodels
def confounder_string_statsmodels(confounders,categorical_cols=['REGION', 'LOCATION']):
    terms = []
    for c in confounders.columns:
        if c in categorical_cols:
            terms.append(f"C({c})")
        else:
            terms.append(c)

    conf_list = " + ".join(terms)
    alpha_dim = len(confounders.columns)
    return conf_list, alpha_dim

conf_list, alpha_dim = confounder_string_statsmodels(confounders)


In [ ]:
## Calculate the adjusted odds ratios for all interested medication and disease pairs.
## Confounding adjustment with L1-regularized logistic regression with optimal shrinkage coefficient determined by hyperparameter tuning with Optuna
store_dict = {}
ix =0
for dz, med in zip(med_dz_pair['Disease'],med_dz_pair['Medication']):
    conf_col =list(confounders.columns)
    feature_list = [dz]+conf_col+[med]
    sel_data = odds_df[feature_list]
    
    X = sel_data.drop(dz, axis=1)  # Features
    y = sel_data[dz]  # Target variable
    
    ## Hyperparameter Tuning to determine the best shrinkage coefficient ##
    def objective(trial):
        shrink_coeff = trial.suggest_float('shrink_coeff', 0.01, 10, log=True)
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        f1_scores = []
        print('Start to Calculate -- Shrink Coefficinet: {}'.format(shrink_coeff))
        for train_index, test_index in skf.split(X,y):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            train_data = X_train.copy()
            train_data[dz] = y_train
            test_data = X_test.copy()
            test_data[dz] = y_test

            stats_model_input = f"{dz} ~ {conf_list} + Q('{med}')"
            shrink_coeff_list= np.full(alpha_dim+1,shrink_coeff) ## applying shrinkange coefficients
            shrink_coeff_list = np.append(shrink_coeff_list,0) ## no shrinkage for medication variable by adding zero
                    
            try:
                logit_model = smf.logit(stats_model_input, data=train_data)
                result = logit_model.fit_regularized(method='l1', alpha=shrink_coeff_list, maxiter=100,  #####
                                                    trim_mode='auto', full_output=True, disp=True, maxfun=100)
                predictions = result.predict(X_test)
                binary_predictions = (predictions >= 0.5).astype(int)
                f1 = f1_score(y_test, binary_predictions)
                f1_scores.append(f1)
                print('Non-singular matrix -- conducted calculation!')
            except Exception as e:
                logging.error("Error occurred with shrink_coeff %s: %s", shrink_coeff, str(e))
                pass

        print('f1_scores:',f1_scores)
        average_f1_score = np.mean(f1_scores)
        print(average_f1_score)
        return average_f1_score
    if __name__ == "__main__":
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=10, n_jobs=-1) 
    
    try:    
        best_params = study.best_params
        best_shrink_coeff = best_params['shrink_coeff']
        print("Best shrink coefficient {}-{}:".format(dz,med), best_shrink_coeff)
    except:
        best_shrink_coeff = 1
        print("Hyperparameter tuning failed {}-{}:".format(dz,med), best_shrink_coeff)
    
    data = sel_data.copy()
    stats_model_input = f"{dz} ~ {conf_list} + Q('{med}')"
    shrink_coeff_list= np.full(alpha_dim+1,best_shrink_coeff)
    shrink_coeff_list = np.append(shrink_coeff_list,0)

    logit_model = smf.logit(stats_model_input, data=data)
    result = logit_model.fit_regularized(method='l1', alpha=shrink_coeff_list, maxiter=100, 
                                    trim_mode='auto', full_output=True, disp=True, maxfun=100)

    med_key = f"Q('{med}')"
    beta = result.params[med_key]
    OR = np.exp(beta)
    pval = result.pvalues[med_key]
    SE = result.bse[med_key]
    OR_LL = np.exp(result.conf_int(alpha=0.05)[0][med_key])
    OR_UL = np.exp(result.conf_int(alpha=0.05)[1][med_key])
    store_dict[ix]={'Disease':dz,'Medication':med,'odds ratio':OR,'p-val':pval,'95% CI (LL)':OR_LL,'95% CI (UL)': OR_UL,'best_shrink_coeff':best_shrink_coeff}
    print('{}th calculation is done: {} and {}.'.format(ix,dz,med))
    print(ix/med_dz_pair.shape[0]*100,'% is done')
    ix+=1
    
adj_odds_table = pd.DataFrame.from_dict(store_dict,'index')


In [10]:
adj_odds_table

,Disease,Medication,odds ratio,p-val,95% CI (LL),95% CI (UL),best_shrink_coeff
0,RDS_Baby,"Ondansetron, Oral",5.550456,0.0,5.217694,5.90444,0.028599


In [11]:
def Benjamini_Hochberg_correction(df, p_value_column,p_value=0.05):
    """
    Benjamini-Hochberg correction for multiple hypothesis testing.
    
    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing p-values.
    p_value_column : str
        Column name of p-values.
    q_value_column : str
        Column name of q-values.
        
    Returns
    -------
    None
    """
    df = df.sort_values(by=p_value_column, ascending=True)
    df = df.reset_index().drop(columns=['index'],axis=1)
    df['k']=df.index+1
    df['m']=df.shape[0]
    df['a']=0.05
    df['B-H critical value']=df['k']*df['a']/df['m']
    df['BH-significance']=(df[p_value_column]<df['B-H critical value'])
    BH_true_df = df[df['BH-significance']==True]
    return BH_true_df

In [12]:
Benjamini_Hochberg_correction(adj_odds_table,'p-val',p_value=0.05)


,Disease,Medication,odds ratio,p-val,95% CI (LL),95% CI (UL),best_shrink_coeff,k,m,a,B-H critical value,BH-significance
0,RDS_Baby,"Ondansetron, Oral",5.550456,0.0,5.217694,5.90444,0.028599,1,1,0.05,0.05,True


In [13]:
Benjamini_Hochberg_correction(adj_odds_table,'p-val',p_value=0.05).to_csv('../0_data/results/adjusted_odds_results.csv',index=False)